In [1]:
import pandas as pd 
import dataparser 
from  math import log
import numpy as np
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re


import folium

pd.set_option('display.max_columns', None)
data=pd.read_csv('_data.csv', index_col=1)

#если цена не в рублях то выбрасываем, цены на доллар и евро сильно меняются, непонятно какая цена в рублях (там всего 27 вариантов)
data=data[data['Цена'].str.contains('руб.')].reset_index()

#убираем всех не из Москвы
data=data[data['Адрес'].str.contains('Москва')].reset_index()

#заполняем пустые места для количества комнат, так как это студии, 1 комната
#фича "Тип" у всех одинаковая, дропаем
data=data.fillna({'Количество комнат':1})\
        .drop('Тип',axis=1)




In [2]:
cdata=dataparser.get_data(data)

/home/ham/elb/d035/real_estate_EDA/dataparser.py:306: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cdata['garbage_chute'] = data['Мусоропровод'].replace({'Да': 1, 'Нет': 0, np.nan: 0}).astype(int)


In [157]:
#для получения координат по адресам

from geopy.geocoders import Nominatim
addarr=cdata['adress']
addrdic={}

def get_metro_station_coordinates(station_name):
    geolocator = Nominatim(user_agent="metro_locator")  

    # Добавим "метро" к запросу для более точного результата
    location = geolocator.geocode(f"{station_name} , Москва")

    if location:
        coordinates = (location.latitude, location.longitude)
        return coordinates
    else:
        print(f"Координаты для станции метро {station_name} не найдены.")
        return None



for i in addarr:
    try:
        if i in addrdic.keys():
            continue
        addrdic[i] = get_metro_station_coordinates(i)
    except:
        pass

with open('coordinates', 'wb') as file:
    pickle.dump(addrdic, file)
